In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.spatial import distance


In [ ]:
# smooth the data before tracking

# read all csv files, then add them to one big df then smooth the data and export  back to csv


In [ ]:
path = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1264_play'
detects = sorted_alpha(path)
numbers = [int((x.split("."))[0].split("_")[1]) for x in detects]

In [ ]:
def process_skeleton_data(csv_file_path):
    # 1. read data
    data = pd.read_csv(csv_file_path)
    temp_data = data.copy()

    
    # 2. drop skeletons with missing neck
    for i in range(0, len(data.columns[1:]), 3):
        neck_value = data[data.columns[1:][i]].loc[1]
        if pd.isna(neck_value) or neck_value == 0:
            temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1)  # Drop skeletons with missing neck
    data = temp_data

    # 3. if no skeletons with neck return None
    if len(data.columns) < 4:  # If not enough skeletons with neck present, return None
        return None, None

    # remove fourth skeleton if detected
    elif len(data.columns) > 10:
        # Drop the fourth skeleton if it has been detected
        data = data.drop(data.columns[10:], axis=1) 

    # 4. initialize the tracking with the first two skeletons and save the neck_points
    detections = []
    neck_keypoints = []
    for col in range(0, len(data.columns[1:]), 3):
        skeleton = data.iloc[:, col:col+3].values
        detections.append(skeleton[:,1:])  # Select only x and y columns
        neck_keypoints.append(skeleton[:,1:][1])  # Assuming neck keypoint is the second row (index 1)
    
    detections = np.array(detections)  # Shape: (num_skeletons, num_keypoints)
    neck_keypoints = np.array(neck_keypoints)  # Shape: (num_skeletons, 2)

    return detections, neck_keypoints


In [ ]:
csv_file_path = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1264_play/frame_3318.csv'
detections, neck_keypoints = process_skeleton_data(csv_file_path)
detections

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

# Initialize tracking variables
tracks = []
frames = []
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')

# Example usage
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int(f.split('_')[-1].split('.')[0]) for f in detects]

for i, number in zip(range(len(detects)), numbers):
    frames.append(number)
    print('frame no:', number)
    
    data = pd.read_csv(os.path.join(path, detects[i]))
    
    if i == 0:  # For the first frame, just get the skeletons, don't do anything
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Initialize track lists for each detection
        for _ in range(len(detections)):
            tracks.append([])

        # Append each detection to its corresponding track list
        for j, detection in enumerate(detections):
            tracks[j].append(detection)

    else:
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Get the last neck positions from the tracks
        last_necks = np.array([track[-1][1] for track in tracks])

        # Fit the KNN model on the last neck positions
        knn.fit(last_necks)
        # Keep track of which detections have been assigned
        assigned_detections = set()

        # Store distances and indices
        distances_indices = []
        # Compute Euclidean distances between each pair of new detections 
        necks_array = np.array(necks)
        num_detections = len(necks_array)
        distances = np.zeros((num_detections, num_detections))


        for j in range(num_detections):
            for k in range(j + 1, num_detections):
                dist = euclidean(necks_array[j], necks_array[k])
                distances[j, k] = dist
                distances[k, j] = dist  # Symmetric matrix
        # Check if distance between the first 2 detections are more than 200
        if num_detections > 1:
            if i <50:
                if  np.any(distances[0:2,0:2]) > 200:
                    # Find the closest track for each new detection
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        distances_indices.append((dist[0][0], index[0][0], j))

                    # Sort detections by distance to the closest track
                    distances_indices.sort()

                    # Assign the closest detections to the tracks
                    for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                        tracks[closest_track_index].append(detections[j])
                        assigned_detections.add(j)

                    # If there are more detections than tracks, create new tracks for the remaining detections
                    for dist, closest_track_index, j in distances_indices[len(tracks):]:
                        new_track_index = len(tracks)
                        tracks.append([])  # Initialize a new track
                        tracks[new_track_index].append(detections[j])
                        assigned_detections.add(j)
                else:
                    relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                    if relative_position == 1:
                        tracks[0].append(detections[0])
                        tracks[1].append(detections[1])
                    else:
                        tracks[0].append(detections[1])
                        tracks[1].append(detections[0])
                    
        elif num_detections == 1:
            if np.any(distances) > 200:
            # this should still do the relative position calculation if distance<200 
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    tracks[index[0][0]].append(detections[j])
                    assigned_detections.add(j)
            else:
                if i <50:
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        tracks[index[0][0]].append(detections[j])
                        assigned_detections.add(j)

                   ## think about this : what if there's only one detection and i>50?  
                else:
                    relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                    if relative_position == 1:
                        tracks[0].append(detections[0])
                        tracks[1].append(detections[1])
                    else:
                        tracks[0].append(detections[1])
                        tracks[1].append(detections[0])



### the case where num_detections > 1 and distances <200  and i < 50 is not handled yet


           




In [ ]:
def relative_position(tracks, necks):
    num_tracks = len(tracks)
    num_necks = len(necks)

    if num_necks == 1:
        neck = necks[0]
        track1, track2 = tracks[:2]

        x_diff = [d1[0][0] - d2[0][0] for d1, d2 in zip(track1[-50:], track2[-50:])]
        y_diff = [d1[0][1] - d2[0][1] for d1, d2 in zip(track1[-50:], track2[-50:])]

        mean_x_diff = np.mean(x_diff)
        mean_y_diff = np.mean(y_diff)

        if abs(mean_x_diff) > abs(mean_y_diff):
            return 1 if neck[0] > (track1[-1][0][0] + track2[-1][0][0]) / 2 else 2
        else:
            return 1 if neck[1] > (track1[-1][0][1] + track2[-1][0][1]) / 2 else 2

    track1, track2 = tracks[:2]
    neck1, neck2 = necks[:2]

    x_diff = [d1[0][0] - d2[0][0] for d1, d2 in zip(track1[-50:], track2[-50:])]
    y_diff = [d1[0][1] - d2[0][1] for d1, d2 in zip(track1[-50:], track2[-50:])]

    mean_x_diff = np.mean(x_diff)
    mean_y_diff = np.mean(y_diff)

    if abs(mean_x_diff) > abs(mean_y_diff):  # is x axis difference greater than y axis difference
        if mean_x_diff > 0:  # what's their relative position on x axis
            return 1 if neck1[0] > neck2[0] else 2  # maintain relative position
        else:
            return 1 if neck1[0] < neck2[0] else 2
    else:  # is y axis difference greater than x axis difference
        if mean_y_diff > 0:  # what's their relative position on y axis
            return 1 if neck1[1] > neck2[1] else 2  # maintain relative position
        else:
            return 1 if neck1[1] < neck2[1] else 2

In [ ]:
### ASPECT RATIO INSTEAD OF RELATIVE POSITION
import numpy as np

import numpy as np

def relative_position(tracks, detections):
    num_tracks = len(tracks)
    num_detections = len(detections)

    if num_detections == 1:
        detection = detections[0]
        track1, track2 = tracks[:2]

        aspect_ratio_track1 = [detection[1][0] / detection[1][1] for detection, _ in track1[-50:]]
        aspect_ratio_track2 = [detection[1][0] / detection[1][1] for detection, _ in track2[-50:]]

        mean_aspect_ratio_track1 = np.mean(aspect_ratio_track1)
        mean_aspect_ratio_track2 = np.mean(aspect_ratio_track2)

        aspect_ratio_detection = detection[1][0] / detection[1][1]

        if abs(aspect_ratio_detection - mean_aspect_ratio_track1) < abs(aspect_ratio_detection - mean_aspect_ratio_track2):
            return 1
        else:
            return 2

    track1, track2 = tracks[:2]
    detection1, detection2 = detections[:2]

    aspect_ratio_track1 = [detection[1][0] / detection[1][1] for detection, _ in track1[-50:]]
    aspect_ratio_track2 = [detection[1][0] / detection[1][1] for detection, _ in track2[-50:]]

    mean_aspect_ratio_track1 = np.mean(aspect_ratio_track1)
    mean_aspect_ratio_track2 = np.mean(aspect_ratio_track2)

    aspect_ratio_detection1 = detection1[1][0] / detection1[1][1]
    aspect_ratio_detection2 = detection2[1][0] / detection2[1][1]

    if abs(aspect_ratio_detection1 - mean_aspect_ratio_track1) < abs(aspect_ratio_detection1 - mean_aspect_ratio_track2):
        return 1
    else:
        return 2

In [ ]:
### USE AREA OF BOUNDING BOX INSTEAD OF ASPECT RATIO

import numpy as np

def relative_position(tracks, detections):
    num_tracks = len(tracks)
    num_detections = len(detections)

    if num_detections == 1:
        detection = detections[0]
        track1, track2 = tracks[:2]

        area_track1 = [detection[1][0] * detection[1][1] for detection, _ in track1[-50:]]
        area_track2 = [detection[1][0] * detection[1][1] for detection, _ in track2[-50:]]

        mean_area_track1 = np.mean(area_track1)
        mean_area_track2 = np.mean(area_track2)

        area_detection = detection[1][0] * detection[1][1]

        if abs(area_detection - mean_area_track1) < abs(area_detection - mean_area_track2):
            return 1
        else:
            return 2

    track1, track2 = tracks[:2]
    detection1, detection2 = detections[:2]

    area_track1 = [detection[1][0] * detection[1][1] for detection, _ in track1[-50:]]
    area_track2 = [detection[1][0] * detection[1][1] for detection, _ in track2[-50:]]

    mean_area_track1 = np.mean(area_track1)
    mean_area_track2 = np.mean(area_track2)

    area_detection1 = detection1[1][0] * detection1[1][1]
    area_detection2 = detection2[1][0] * detection2[1][1]

    if abs(area_detection1 - mean_area_track1) < abs(area_detection1 - mean_area_track2):
        return 1
    else:
        return 2

In [ ]:
### COMBINE AREA AND ASPECT RATIO 
import numpy as np

def relative_position(tracks, detections):
    def calculate_bounding_box(keypoints):
        if keypoints is None or len(keypoints) == 0 or not isinstance(keypoints[0], (list, np.ndarray)):
            print(f"Invalid keypoints format: {keypoints}")
            raise ValueError("Invalid keypoints format")
        x_coordinates = []
        y_coordinates = []
        for point in keypoints:
            if isinstance(point, (list, np.ndarray)) and len(point) == 2:
                if not np.isnan(point[0]) and not np.isnan(point[1]):
                    x_coordinates.append(point[0])
                    y_coordinates.append(point[1])
            else:
                print(f"Invalid point format: {point}")
        if len(x_coordinates) == 0 or len(y_coordinates) == 0:
            return 0, 0  # Return zero width and height if all coordinates are NaN
        x_min, x_max = min(x_coordinates), max(x_coordinates)
        y_min, y_max = min(y_coordinates), max(y_coordinates)
        width = x_max - x_min
        height = y_max - y_min
        return width, height
    
    def calculate_mean_area_aspect_ratio(track):
        areas = []
        aspect_ratios = []
        for detection, _ in track[-50:]:
            width, height = calculate_bounding_box(detection)
            areas.append(width * height)
            aspect_ratios.append(width / height if height != 0 else 0)
        return np.mean(areas), np.mean(aspect_ratios)

    def calculate_score(detection, mean_area, mean_aspect_ratio):
        width, height = calculate_bounding_box(detection)
        area = width * height
        aspect_ratio = width / height if height != 0 else 0
        return abs(area - mean_area) + abs(aspect_ratio - mean_aspect_ratio)

    track1, track2 = tracks[:2]
    mean_area_track1, mean_aspect_ratio_track1 = calculate_mean_area_aspect_ratio(track1)
    mean_area_track2, mean_aspect_ratio_track2 = calculate_mean_area_aspect_ratio(track2)

    if len(detections) == 1:
        detection = detections[0]
        score_track1 = calculate_score(detection, mean_area_track1, mean_aspect_ratio_track1)
        score_track2 = calculate_score(detection, mean_area_track2, mean_aspect_ratio_track2)
        return 1 if score_track1 < score_track2 else 2

    detection1, detection2 = detections[:2]
    score_track1_detection1 = calculate_score(detection1, mean_area_track1, mean_aspect_ratio_track1)
    score_track2_detection1 = calculate_score(detection1, mean_area_track2, mean_aspect_ratio_track2)
    score_track1_detection2 = calculate_score(detection2, mean_area_track1, mean_aspect_ratio_track1)
    score_track2_detection2 = calculate_score(detection2, mean_area_track2, mean_aspect_ratio_track2)

    if score_track1_detection1 + score_track2_detection2 < score_track2_detection1 + score_track1_detection2:
        return 1
    else:
        return 2

In [ ]:
### COMBINE multiple bbox features
import numpy as np

def relative_position(tracks, detections):
    def calculate_bounding_box(keypoints):
        if keypoints is None or len(keypoints) == 0 or not isinstance(keypoints[0], (list, np.ndarray)):
            print(f"Invalid keypoints format: {keypoints}")
            raise ValueError("Invalid keypoints format")
        x_coordinates = []
        y_coordinates = []
        for point in keypoints:
                x_coordinates.append(point[0])
                y_coordinates.append(point[1])

        if len(x_coordinates) == 0 or len(y_coordinates) == 0:
            return 0, 0, 0, 0  # Return zero width and height if all coordinates are NaN
        x_min, x_max = np.nanmin(x_coordinates), np.nanmax(x_coordinates)
        y_min, y_max = np.nanmin(y_coordinates), np.nanmax(y_coordinates)
        width = x_max - x_min
        height = y_max - y_min
        return x_min, y_min, x_max, y_max, width, height


    def calculate_iou(box1, box2):
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2

        xi1 = max(x1_min, x2_min)
        yi1 = max(y1_min, y2_min)
        xi2 = min(x1_max, x2_max)
        yi2 = min(y1_max, y2_max)
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

        box1_area = (x1_max - x1_min) * (y1_max - y1_min)
        box2_area = (x2_max - x2_min) * (y2_max - y2_min)
        union_area = box1_area + box2_area - inter_area

        return inter_area / union_area if union_area != 0 else 0

    def calculate_center(keypoints):
        if keypoints is None or len(keypoints) == 0 or not isinstance(keypoints[0], (list, np.ndarray)):
            print(f"Invalid keypoints format: {keypoints}")
            raise ValueError("Invalid keypoints format")
        x_coordinates = []
        y_coordinates = []
        for point in keypoints:
            if isinstance(point, (list, np.ndarray)) and len(point) == 2:
                if not np.isnan(point[0]) and not np.isnan(point[1]):
                    x_coordinates.append(point[0])
                    y_coordinates.append(point[1])
            else:
                print(f"Invalid point format center: {point}")
        if len(x_coordinates) == 0 or len(y_coordinates) == 0:
            return 0, 0  # Return zero width and height if all coordinates are NaN
        center_x = np.mean(x_coordinates)
        center_y = np.mean(y_coordinates)
        return center_x, center_y

    def calculate_diagonal(width, height):
        return np.sqrt(width**2 + height**2)

    def calculate_velocity(center1, center2):
        return np.sqrt((center2[0] - center1[0])**2 + (center2[1] - center1[1])**2)

    def calculate_mean_features(track):
        areas = []
        aspect_ratios = []
        centers = []
        diagonals = []
        velocities = []
        for i in range(1, len(track[-50:])):
            detection, _ = track[-50:][i]
            prev_detection, _ = track[-50:][i-1]
            _, _, _, _, width, height = calculate_bounding_box(detection)
            prev_center = calculate_center(prev_detection)
            center = calculate_center(detection)
            areas.append(width * height)
            aspect_ratios.append(width / height if height != 0 else 0)
            centers.append(center)
            diagonals.append(calculate_diagonal(width, height))
            velocities.append(calculate_velocity(prev_center, center))
        return np.mean(areas), np.mean(aspect_ratios), np.mean(centers, axis=0), np.mean(diagonals), np.mean(velocities)

    def calculate_score(detection, mean_area, mean_aspect_ratio, mean_center, mean_diagonal, mean_velocity, last_detection):
        x_min, y_min, x_max, y_max, width, height = calculate_bounding_box(detection)
        area = width * height
        aspect_ratio = width / height if height != 0 else 0
        center = calculate_center(detection)
        diagonal = calculate_diagonal(width, height)
        velocity = calculate_velocity(calculate_center(last_detection), center)
        iou = calculate_iou((x_min, y_min, x_max, y_max), calculate_bounding_box(last_detection)[:4])
        return (abs(area - mean_area) + abs(aspect_ratio - mean_aspect_ratio) +
                np.linalg.norm(np.array(center) - np.array(mean_center)) + abs(diagonal - mean_diagonal) +
                abs(velocity - mean_velocity) - iou)

    track1, track2 = tracks[:2]
    mean_features_track1 = calculate_mean_features(track1)
    mean_features_track2 = calculate_mean_features(track2)

    if len(detections) == 1:
        detection = detections[0]
        score_track1 = calculate_score(detection, *mean_features_track1, track1[-1][0])
        score_track2 = calculate_score(detection, *mean_features_track2, track2[-1][0])
        return 1 if score_track1 < score_track2 else 2

    detection1, detection2 = detections[:2]
    score_track1_detection1 = calculate_score(detection1, *mean_features_track1, track1[-1][0])
    score_track2_detection1 = calculate_score(detection1, *mean_features_track2, track2[-1][0])
    score_track1_detection2 = calculate_score(detection2, *mean_features_track1, track1[-1][0])
    score_track2_detection2 = calculate_score(detection2, *mean_features_track2, track2[-1][0])

    if score_track1_detection1 + score_track2_detection2 < score_track2_detection1 + score_track1_detection2:
        return 1
    else:
        return 2

In [ ]:
def calculate_bounding_box(keypoints):
    x_coordinates = [point[0] for point in keypoints if not np.isnan(point[0])]
    y_coordinates = [point[1] for point in keypoints if not np.isnan(point[1])]
    if not x_coordinates or not y_coordinates:
        return 0, 0  # Return zero width and height if all coordinates are NaN
    x_min, x_max = min(x_coordinates), max(x_coordinates)
    y_min, y_max = min(y_coordinates), max(y_coordinates)
    width = x_max - x_min
    height = y_max - y_min
    return width, height
    

In [ ]:
from scipy.spatial.distance import euclidean
import os
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import linear_sum_assignment

# Initialize tracking variables
tracks = []
frames = []
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
threshold = 80
# Example usage
path = '/Users/andrei-macpro/Documents/Data/pose/play_openpose/1264_play'
detects = sorted_alpha(path)
numbers = [int(f.split('_')[-1].split('.')[0]) for f in detects]


for i, number in zip(range(len(detects)), numbers):
    frames.append(number)
    print('frame no:', number)
    
    data = pd.read_csv(os.path.join(path, detects[i]))
    
    if i == 0:  # For the first frame, just get the skeletons, don't do anything
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Initialize track lists for each detection
        for _ in range(len(detections)):
            tracks.append([])

        # Append each detection to its corresponding track list with the frame number
        for j, detection in enumerate(detections):
            tracks[j].append((detection, number))

    else:
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))

        # Check if there are valid neck joints
        if necks is None or len(necks) == 0 or all(neck is None for neck in necks):
            continue
        
        # Get the last neck positions from the tracks
        last_necks = np.array([track[-1][0][1] for track in tracks])

        # Fit the KNN model on the last neck positions
        knn.fit(last_necks)
        # Keep track of which detections have been assigned
        assigned_detections = set()
        # Store distances and indices
        distances_indices = []
        # Compute Euclidean distances between each pair of new detections 
        necks_array = np.array(necks)
        num_detections = len(necks_array)
        distances = np.zeros((num_detections, num_detections))

        for j in range(num_detections):
            for k in range(j + 1, num_detections):
                dist = euclidean(necks_array[j], necks_array[k])
                distances[j, k] = dist
                distances[k, j] = dist  # Symmetric matrix

        # Handle the case where there is only one detection
        if num_detections == 1:
            if len(tracks) == 1:
                tracks[0].append((detections[0], number))

            else:
                # Assign the single detection to the closest track
                dist, index = knn.kneighbors([necks[0]])
                tracks[index[0][0]].append((detections[0], number))

        else:
            # Check if distance between the first 2 detections is less than 80
            if i > 50 and np.any(distances[0,1] < threshold):
                if len(tracks) == 1:
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        distances_indices.append((dist[0][0], index[0][0], j))
                        # Sort detections by distance to the closest track
                        distances_indices.sort()
                        # Assign the closest detections to the tracks
                        for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                            tracks[closest_track_index].append((detections[j], number))
                            assigned_detections.add(j)
                else:
                    relative_pos = relative_position(tracks, detections)
                    

                    if relative_pos == 1:
                        tracks[0].append((detections[0], number))
                        tracks[1].append((detections[1], number))
                    else:
                        tracks[0].append((detections[1], number))
                        tracks[1].append((detections[0], number))
            else:
                # Find the closest track for each new detection
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    distances_indices.append((dist[0][0], index[0][0], j))


                # Sort detections by distance to the closest track
                distances_indices.sort()
                if len(detections) == 3 and len(tracks) == 2:
                     # Assign the closest detections to the tracks using k-NN
                    assigned_detections = set()
                    for dist, closest_track_index, j in distances_indices[:2]:
                        tracks[closest_track_index].append((detections[j], number))
                        assigned_detections.add(j)

          
       
                else:

                    # Assign the closest detections to the tracks
                    for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                        tracks[closest_track_index].append((detections[j], number))
                        assigned_detections.add(j)

                 

In [ ]:
import pandas as pd

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]

# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Set the frame number, keypoint, and track as a multi-index
df_tracks.set_index(['Frame', 'Keypoint', 'Track'], inplace=True)

# Display the DataFrame
print(df_tracks)

In [ ]:
import pandas as pd

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]

# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Pivot the DataFrame to have 'Track' as columns and keypoints as rows
df_pivot = df_tracks.pivot_table(index=['Frame', 'Keypoint'], columns='Track', values=['X', 'Y'])

# Flatten the multi-level columns
df_pivot.columns = [f'{coord}_Track{track}' for coord, track in df_pivot.columns]

# Reset the index to make 'Frame' and 'Keypoint' columns
df_pivot.reset_index(inplace=True)

# Display the DataFrame
print(df_pivot)

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

def interpolate_missing_skeletons(tracks):
    """
    Interpolates missing skeletons in the tracks and returns a DataFrame.

    Parameters:
    tracks (list of list of tuples): The tracks where each track is a list of (detection, frame_number) tuples.

    Returns:
    pd.DataFrame: The DataFrame with interpolated skeletons, with tracks as columns.
    """
    data = []

    for track_index, track in enumerate(tracks):
        for detection, frame_number in track:
            for keypoint_index, (x, y) in enumerate(detection):
                data.append({'Frame': frame_number, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

    # Create a DataFrame from the data
    df_tracks = pd.DataFrame(data)

    # Ensure all frames are present
    all_frames = pd.DataFrame({'Frame': range(df_tracks['Frame'].min(), df_tracks['Frame'].max() + 1)})
    df_tracks = all_frames.merge(df_tracks, on='Frame', how='left')

    # Pivot the DataFrame to have 'Frame' and 'Keypoint' as index, and 'Track' as columns
    df_pivot = df_tracks.pivot_table(index=['Frame', 'Keypoint'], columns='Track', values=['X', 'Y'])

    # Interpolate missing values using linear interpolation
    df_interpolated = df_pivot.groupby(level='Keypoint').apply(lambda group: group.interpolate(method='linear', limit_direction='both'))

    # Apply Savitzky-Golay filter for smoothing
    def apply_savgol_filter(group):
        return group.apply(lambda col: savgol_filter(col, window_length=11, polyorder=3) if col.notna().sum() > 11 else col)

    df_smoothed = df_interpolated.groupby(level='Keypoint').apply(apply_savgol_filter)

    # Convert the Track level to integer
    df_smoothed.columns = pd.MultiIndex.from_tuples([(coord, int(track)) for coord, track in df_smoothed.columns])

    # Flatten the multi-level columns
    df_smoothed.columns = [f'{coord}_Track{track}' for coord, track in df_smoothed.columns]

    # Reset the index to make 'Frame' and 'Keypoint' columns
    df_smoothed.reset_index(inplace=True)

    return df_smoothed


interpolated_df = interpolate_missing_skeletons(tracks)
print(interpolated_df)

In [ ]:
import pandas as pd

## let's see how the interpolated tracks look like in a dataframe 

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]

# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Pivot the DataFrame to have 'Track' as columns and keypoints as rows
df_pivot = df_tracks.pivot_table(index=['Frame', 'Keypoint'], columns='Track', values=['X', 'Y'])

# Flatten the multi-level columns
df_pivot.columns = [f'{coord}_Track{track}' for coord, track in df_pivot.columns]

# Reset the index to make 'Frame' and 'Keypoint' columns
df_pivot.reset_index(inplace=True)

# Display the DataFrame
print(df_pivot)

In [ ]:
interpolated_df

In [ ]:
import pandas as pd

## let's see how the interpolated tracks look like in a dataframe 

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]
tracks = interpolated_tracks
# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Pivot the DataFrame to have 'Track' as columns and keypoints as rows
df_pivot = df_tracks.pivot_table(index=['Frame', 'Keypoint'], columns='Track', values=['X', 'Y'])

# Flatten the multi-level columns
df_pivot.columns = [f'{coord}_Track{track}' for coord, track in df_pivot.columns]

# Reset the index to make 'Frame' and 'Keypoint' columns
df_pivot.reset_index(inplace=True)

# Display the DataFrame
print(df_pivot)

In [ ]:
df_pivot.to_csv('interpolated_output_all_frames.csv')

In [ ]:
interpolated_df.to_csv('interpolated_output.csv')

## now draw bounding boxes before imputation 

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1264_play.mp4'
cap = cv2.VideoCapture(video_path)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_path = '/Users/andrei-macpro/Documents/Data/videos/1264_play_annotated_multiple_.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Flag to toggle frame number display
show_frame_number = True

# Iterate over the video frames
frame_count = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the rows for the current frame
    frame_data = df_pivot[df_pivot['Frame'] == frame_count]

    # Iterate over each track
    for track in range(len(tracks)):
        # Get the keypoints for the current track
        keypoints = frame_data[[f'X_Track{track}', f'Y_Track{track}']].values

        # Choose a color for the current track
        color = colors[track % len(colors)]

        # Draw bounding boxes around the keypoints and display track number
        for x, y in keypoints:
            if not pd.isna(x) and not pd.isna(y):
                cv2.rectangle(frame, (int(x) - 5, int(y) - 5), (int(x) + 5, int(y) + 5), color, 2)
                cv2.putText(frame, f'Track {track}', (int(x) + 10, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    # Display the frame number if the flag is set
    if show_frame_number:
        cv2.putText(frame, f'Frame {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)



    # Write the frame to the output video
    out.write(frame)

    frame_count += 1

# Release the video capture object and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# save df_pivot to a csv file
df_pivot.to_csv('output.csv')

## after imputation

In [ ]:
# next up: interpolation + see how it performs on harder ones
tracks = interpolated_tracks

video_path = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1264_play.mp4'
cap = cv2.VideoCapture(video_path)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_path = '/Users/andrei-macpro/Documents/Data/videos/1264_play_annotated_imputation.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Flag to toggle frame number display
show_frame_number = True

# Iterate over the video frames
frame_count = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the rows for the current frame
    frame_data = df_pivot[df_pivot['Frame'] == frame_count]

    # Iterate over each track
    for track in range(len(tracks)):
        # Get the keypoints for the current track
        keypoints = frame_data[[f'X_Track{track}', f'Y_Track{track}']].values

        # Choose a color for the current track
        color = colors[track % len(colors)]

        # Draw bounding boxes around the keypoints and display track number
        for x, y in keypoints:
            if not pd.isna(x) and not pd.isna(y):
                cv2.rectangle(frame, (int(x) - 5, int(y) - 5), (int(x) + 5, int(y) + 5), color, 2)
                cv2.putText(frame, f'Track {track}', (int(x) + 10, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    # Display the frame number if the flag is set
    if show_frame_number:
        cv2.putText(frame, f'Frame {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)



    # Write the frame to the output video
    out.write(frame)

    frame_count += 1

# Release the video capture object and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/videos/play_videos/1264_play.mp4'
cap = cv2.VideoCapture(video_path)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_path = '/Users/andrei-macpro/Documents/Data/videos/1264_play_annotated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Flag to toggle frame number display
show_frame_number = True

# Iterate over the video frames
frame_count = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the rows for the current frame
    frame_data = interpolated_df[interpolated_df['Frame'] == frame_count]

    # Iterate over each track
    for track in range(len(tracks)):
        # Get the keypoints for the current track
        keypoints = frame_data[[f'X_Track{track}', f'Y_Track{track}']].values

        # Choose a color for the current track
        color = colors[track % len(colors)]

        # Draw bounding boxes around the keypoints and display track number
        for x, y in keypoints:
            if not pd.isna(x) and not pd.isna(y):
                cv2.rectangle(frame, (int(x) - 5, int(y) - 5), (int(x) + 5, int(y) + 5), color, 2)
                cv2.putText(frame, f'Track {track}', (int(x) + 10, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    # Display the frame number if the flag is set
    if show_frame_number:
        cv2.putText(frame, f'Frame {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Write the frame to the output video
    out.write(frame)

    # Increment the frame count
    frame_count += 1

# Release the video capture and writer objects
cap.release()
out.release()